In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from PIL import Image
import pandas as pd, torch, gc
import torch
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
import ast

import nbimporter
from test_MLLMs.ipynb import mllm_testing

SyntaxError: invalid syntax (701316962.py, line 11)

In [ ]:
# Check if ground truth colors are influenced differ if dummy images are used
def get_model_color_priors(df, processor, model):
    batch_size = 1
    results = []

    for i in tqdm(range(0, len(df), batch_size), desc="Running predictions"):
        batch_df = df.iloc[i:i + batch_size].copy()
        with torch.inference_mode():

            df_dummy = mllm_testing(batch_df, processor, model, most="True", dummy_image=True)
            df_dummy = df_dummy.rename(columns={"predicted_color": "model_prior_dummy"})

            df_real = mllm_testing(batch_df, processor, model, most="True", dummy_image=False)
            df_real = df_real.rename(columns={"predicted_color": "model_prior"})

            df_merged = pd.concat([df_dummy, df_real[["model_prior"]]], axis=1)

        results.append(df_merged)
        del df_merged
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        gc.collect()

    ground_truth_df = pd.concat(results, ignore_index=True)
    out_path = data_folder / "model_color_priors.csv"
    ground_truth_df.to_csv(out_path, index=False)
    display(ground_truth_df[['object', 'correct_answer', 'model_prior_dummy', 'model_prior']])

In [ ]:
ground_truth_df = pd.read_csv(data_folder / "model_color_priors.csv")
display(ground_truth_df[['object', 'correct_answer', 'model_prior_dummy', 'model_prior']])

In [ ]:
ground_truth_df["diff_dummy_vs_real"] = ground_truth_df["model_prior_dummy"].str.lower() != ground_truth_df["model_prior"].str.lower()
print(f"There are {ground_truth_df[ground_truth_df['diff_dummy_vs_real']].shape[0]} rows where the predicted color differs between dummy and real images.")

ground_truth_df["correct_answer"] = ground_truth_df["correct_answer"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
ground_truth_df["color_in_gt"] = ground_truth_df.apply(
    lambda r: r["model_prior"].lower() in [c.lower() for c in r["correct_answer"]],
    axis=1
)

In [ ]:
# Check where model specific priors differ from correct answer from Visual Counterfact
print(f"There are {ground_truth_df[~ground_truth_df['color_in_gt']].shape[0]} rows where the model color prior is NOT in the ground truth answers from Visual Counterfact.")
ground_truth_df[~ground_truth_df["color_in_gt"]][["object", "correct_answer", "model_prior"]]

model_priors = ground_truth_df["model_prior"].unique()
print(f"Model color priors: {model_priors}")

In [ ]:
# Drop rows with silver, gold, white clear ground truth answers
colors_to_exclude = ["silver", "gold", "white", "clear"]
ground_truth_df = ground_truth_df[
    ~ground_truth_df["model_prior"].isin(colors_to_exclude)
]
print(f"We are left with {ground_truth_df.shape[0]} rows after excluding colors: {colors_to_exclude}")

In [ ]:
# Replace Visial Counterfact correct answers with model specific priors
df = df.merge(
    ground_truth_df[["object", "model_prior"]],
    on="object",
    how="inner"  # <-- ensures only shared objects remain
)
df["correct_answer"] = df["model_prior"]
df = df.drop(columns=["model_prior"])
display(df)